In [1]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

Empresas Programas por empresa y empresas desde account

In [2]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(8844, 7)

In [3]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(8846, 5)

In [4]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas.shape

(8852, 12)

In [5]:
Programas_pacientes = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_pacientes = Programas_pacientes[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]


fin empresas y comienzo de rescate de pacientes


In [6]:
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1569936, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [7]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF.shape

(1284626, 4)

Union de pacientes programas con las empresas

In [8]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_pacientes,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

(1284626, 12)

In [9]:
empresas_unidas = pd.merge(
    left=empresas_unidas,
    right=Pacientes_SF,
    how="left",
    left_on="Paciente__c",
    right_on="Id Account Paciente",
)
empresas_unidas.shape

(1284626, 21)

In [10]:
empresas_unidas = empresas_unidas.rename(
    columns={
        "Id": "Id Pacientes Programas"       
    }
)
empresas_unidas = empresas_unidas[
    [
        "Id Pacientes Programas",
        "Paciente__c",
        "Programas_por_Empresas__c",
        'Id Account Empresa',
        'Nombre Empresa',
        'Holding',
        'Campaña',
        'Empleador',
        'Poliza',
        'Id Account Paciente',
        'Rut',
        'Nombre',
        'Ocupacion',
        'Cargo',
        'Departamento',
        'Sexo',
        'Telefono',
        'Tipo de Documento',
        'Codigo EMP',
        'Codigo A'


    ]]
empresas_unidas.head()

,Id Pacientes Programas,Paciente__c,Programas_por_Empresas__c,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Id Account Paciente,Rut,Nombre,Ocupacion,Cargo,Departamento,Sexo,Telefono,Tipo de Documento,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,0018c00002amLbpAAE,224488556,Genesis Antonia Gutierrez Abarca,None,None,None,Mujer,967786319,RUT,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbqAAE,113376066,Claudia Silva,None,None,None,Mujer,None,RUT,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbrAAE,79476161,Maria Lorenza Garay,None,None,None,Mujer,None,RUT,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbsAAE,181854227,Estolaza Vidal Jean Patrick Moises,None,None,None,Hombre,953757757,RUT,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLc2AAE,126899823,Rivas Correa Claudia,None,None,None,Mujer,982058845,RUT,EMP6274,A-0353


Terminado la visualizacion de las empresas y pacientes que se muestran en los reportes


Descarga desde Citas de Servicios

In [11]:
# Citas de Servicio
results = sf.query_all(
    """
    SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,Tipo_de_la_cita__c from ServiceAppointment
    """
)


Citas_de_servicio = pd.DataFrame(results["records"])
Citas_de_servicio['Empresas'] = np.nan
#Citas_de_servicio['RUT__c'].map(pacientessf_empresas.set_index('IdentificationId__pc_x',verify_integrity=False)['Name']) 
Citas_de_servicio['Empresas Id'] = np.nan
Citas_de_servicio = Citas_de_servicio[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id',        
        "AccountId",
        'Clinica__c'
        
    ]
]
Citas_de_servicio.shape


(8965, 8)

In [12]:
# Citas de Servicio Orientacion
results = sf.query_all(
    """
    SELECT Id, AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,Tipo_de_la_cita__c,AppointmentNumber from ServiceAppointment
    """
)
Citas_de_servicio2 = pd.DataFrame(results["records"])
Citas_de_servicio2['Clinica__c'] = 'Orientación Médica'
Citas_de_servicio2['Empresas'] = np.nan
Citas_de_servicio2['Empresas Id'] = np.nan
Citas_de_servicio2 = Citas_de_servicio2[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id',       
        "AccountId",
        'Clinica__c',
        
    ]
]
Citas_de_servicio2.shape


(8965, 8)

In [13]:
#unir dos citas
Gran_Citas_deservicio = pd.concat([Citas_de_servicio, Citas_de_servicio2], axis=0)
Gran_Citas_deservicio = Gran_Citas_deservicio.rename(
    columns={
        "AppointmentNumber": "Codigo Ficha",
        "RUT__c": "Rut",
        "Empresas": "Empleador",
        "CreatedDate": "Fecha",
        "Tipo_de_la_cita__c": "Canal",
        "Empresas Id": "Cod. Empresa",
        "AccountId": "Id Paciente",        
        'Clinica__c':'Clínica'
    }
)

Gran_Citas_deservicio.shape

(17930, 8)

In [14]:
#Clinica_deporte
results = sf.query_all(
    """
   SELECT id,Name,Rut__c,Nombre_Empresa__c,CreatedDate,Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Deporte__c
     """
)
ficha_deporte = pd.DataFrame(results["records"])

ficha_deporte = ficha_deporte[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_deporte['clinica'] = 'Deporte'
ficha_deporte = ficha_deporte.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'clinica':'Clínica'
    }
)
ficha_deporte.shape


(1078, 8)

In [15]:
# Clinica Nutricion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Clinica_Nutrici_n__c 
     """
)
ficha_nutricion = pd.DataFrame(results["records"])

ficha_nutricion = ficha_nutricion[
    [
        "Name",
        "RUT__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_nutricion["clinica"] = "Nutrición"

ficha_nutricion = ficha_nutricion.rename(
    columns={
        "Name": "Codigo Ficha",
        "RUT__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)


ficha_nutricion.shape

(2429, 8)

In [16]:
#Clinica AppSex
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_ApsSex__c 
     """
)
ficha_Apssex = pd.DataFrame(results["records"])

ficha_Apssex = ficha_Apssex[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Apssex['clinica'] = 'Asesoría Psicologíca en Salud Sexual'
ficha_Apssex = ficha_Apssex.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)

ficha_Apssex.shape

(5, 8)

In [17]:
# Clinica Psicologia
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId,Motivo_de_consulta_manifiesto__c from Ficha_Cl_nica_Asistencia_Emocional__c 
     """
)
ficha_Psicologia = pd.DataFrame(results["records"])

ficha_Psicologia = ficha_Psicologia[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "Motivo_de_consulta_manifiesto__c",
    ]
]
ficha_Psicologia["clinica"] = "Psicología"
ficha_Psicologia = ficha_Psicologia.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "Motivo_de_consulta_manifiesto__c": "Motivo de consulta",
    }
)
ficha_Psicologia.shape

(1700, 9)

In [18]:
#Clinica Sueño
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_del_Sue_o__c 
     """
)
ficha_Sueno = pd.DataFrame(results["records"])
ficha_Sueno = ficha_Sueno[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Sueno['clinica'] = 'Clínica Del Sueño'
ficha_Sueno = ficha_Sueno.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Sueno.shape

(241, 8)

In [19]:
#Clinica PAciente Cronico
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c  
     """
)
ficha_Cronico = pd.DataFrame(results["records"])
ficha_Cronico = ficha_Cronico[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Cronico['clinica'] = 'Pacientes Crónicos'
ficha_Cronico = ficha_Cronico.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Cronico.shape

(975, 8)

In [20]:
#Clinica Telemedicina
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Telemedicina__c   
     """
)
ficha_Telemedicina = pd.DataFrame(results["records"])
ficha_Telemedicina = ficha_Telemedicina[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Telemedicina['clinica'] = 'Telemedicina'
ficha_Telemedicina = ficha_Telemedicina.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'clinica':'Clínica'
    }
)
ficha_Telemedicina.shape

(2843, 8)

In [21]:
#Clinica Veterinaria
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Veterinaria__c    
     """
)
ficha_Veterinaria = pd.DataFrame(results["records"])
ficha_Veterinaria = ficha_Veterinaria[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Veterinaria['clinica'] = 'Veterinaria'
ficha_Veterinaria = ficha_Veterinaria.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Veterinaria.shape

(23, 8)

In [22]:
#Clinica Orientacion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Orientaci_n_Medica__c     
     """
)
ficha_Orientacion = pd.DataFrame(results["records"])
ficha_Orientacion = ficha_Orientacion[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Orientacion['clinica'] = 'Orientación Médica'
ficha_Orientacion = ficha_Orientacion.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Orientacion.shape

(35, 8)

In [23]:
#Clinica Front
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Front__c     
     """
)
ficha_Front = pd.DataFrame(results["records"])

ficha_Front = ficha_Front[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Front['clinica'] = 'Orientación Médica'
ficha_Front = ficha_Front.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Front.shape

(401, 8)

In [24]:
#Clinica_general
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c,Account__c,   Programa__c   from Ficha_Clinica_General__c 
     """
)
ficha_general = pd.DataFrame(results["records"])

ficha_general = ficha_general[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'Programa__c'
        
    ]
]

ficha_general = ficha_general.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'Programa__c':'Clínica'
    }
)
ficha_general.shape


(1144, 8)

In [25]:
#Clinica Ingreso deportivo
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Ingreso_Deportivo__c    
     """
)
ficha_Ingreso_dep = pd.DataFrame(results["records"])

ficha_Ingreso_dep = ficha_Ingreso_dep[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Ingreso_dep['clinica'] = 'Ingreso Deporte Seguro'
ficha_Ingreso_dep = ficha_Ingreso_dep.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Ingreso_dep.shape

(243, 8)

In [26]:
Clinicas = pd.concat([ficha_Front, ficha_Ingreso_dep,ficha_Orientacion,ficha_Veterinaria,ficha_Telemedicina,ficha_Cronico,ficha_Sueno,ficha_Psicologia,ficha_Apssex,ficha_nutricion,ficha_deporte,ficha_general], axis=0)
Clinicas.shape

(11117, 9)

In [27]:
Reortes_general = pd.concat([Clinicas,Gran_Citas_deservicio ], axis=0)
Reortes_general.shape

(29047, 9)

In [28]:
Reortes_general["Dia"] = pd.to_datetime(Reortes_general["Fecha"]).dt.day

Reortes_general["Mes"] = pd.to_datetime(Reortes_general["Fecha"]).dt.month

Reortes_general["semana"] = pd.to_datetime(Reortes_general["Fecha"]).dt.month_name()

Reortes_general["Año"] = pd.to_datetime(Reortes_general["Fecha"]).dt.year


# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
Reortes_general["Fecha_chilena"] = pd.to_datetime(
    Reortes_general["Fecha"], dayfirst=True, yearfirst=False, exact=True
)
Reortes_general["Fecha_chilena2"] = Reortes_general["Fecha_chilena"].dt.strftime(
    "%d-%m-%Y"
)

clinicas_ = {
    "Clínica de Nutrición": "Nutrición",
    "Clínica de Deporte": "Deporte",
    "Clínica de Psicología": "Psicología",
    "Clínica Ingreso Deportivo Seguro": "Deporte",
    "Clínica del Sueño": "Clínica Del Sueño",
    "Clínica Seguimiento Pacientes Crónicos": "Pacientes Crónicos",
    "Orientación Médica": "Orientación Médica",
    "Asesoría Deportiva": "Deporte",
    "Asistencia Emocional": "Psicología",
    "Asistencia Nutricional": "Nutrición",
    "Clínica APSSex": "Asesoría Psicologíca en Salud Sexual",
    "Clínica Del Sueño": "Clínica Del Sueño",
    "Clínica Veterinaria": "Veterinaria",
    "Ingreso Deportivo Seguro": "Deporte",
    "Clínica Mis Primeros Pasos": "Mis Primeros Pasos",
}
Reortes_general.replace({"Clínica": clinicas_}, inplace=True)

Canal = {
    "Phone": "Teléfono",
    "Video": "Videollamada",
    "Entrante - Chat": "Entrante - Chat Online",
}
Reortes_general.replace({"Canal": Canal}, inplace=True)


Reortes_general["Fecha_chilena"] = Reortes_general["Fecha_chilena"].dt.tz_localize(None)
Reortes_general.Rut.fillna("0", inplace=True)



In [29]:
Reportes_con_empresas = pd.merge(
    left=Reortes_general,
    right=empresas_unidas,
    how="left",
    left_on="Id Paciente",
    right_on="Id Account Paciente",
)


Reportes_con_empresas.shape

(32613, 35)

In [30]:
Reportes_con_empresas['Rut_x'].str.upper()
Reportes_con_empresas['Rut_y'].str.upper()

Reportes_con_empresas.Empleador_x.fillna(Reportes_con_empresas['Nombre Empresa'], inplace=True)

Salida de datos

In [31]:
Reportes_con_empresas.to_excel(
    "C:/Users/jmartinez/Documents/reportes_completos.xlsx", index=False
)


Programas_pacientes.to_excel("C:/Users/jmartinez/Documents/Empresas.xlsx", index=False)
reportes = Reportes_con_empresas[
    [
        "Codigo Ficha",
        "Rut_x",
        "Nombre",
        "Sexo",
        "Ocupacion",
        "Cargo",
        "Departamento",
        "Campaña",
        "Holding",
        "Empleador_x",
        "Poliza",
        "Fecha",
        "Fecha_chilena",
        "Fecha_chilena2",
        "Dia",
        "Mes",
        "Año",
        "Canal",
        "Clínica",
        "Motivo de consulta",
        "Nombre Empresa",
        "Codigo EMP",
    ]
]
reportes = reportes.rename(
    columns={
        "Rut_x": "Rut",
        "Empleador_x": "Empleador"
        
    }
)
reportes.to_excel("C:/Users/jmartinez/Documents/reportes.xlsx", index=False)



In [32]:
general_todo = reportes[(reportes["Empleador"] == reportes["Nombre Empresa"])]
general_todo.to_excel("C:/Users/jmartinez/Documents/general.xlsx", index=False)

Banco Estado

In [33]:
Banco_Estado = reportes[
    (reportes["Codigo EMP"] == "EMP5906")
    | (reportes["Codigo EMP"] == "EMP5907")
    | (reportes["Codigo EMP"] == "EMP6490")
    | (reportes["Codigo EMP"] == "EMP6754")
    | (reportes["Codigo EMP"] == "EMP6837")
    | (reportes["Codigo EMP"] == "EMP6838")
]
Banco_Estado.to_excel('F:/Reportes SF/Banco Estado Metlife/Banco_Estado_Base.xlsx', index=False)


ZURICH

In [34]:
Zurich = reportes[
    (reportes["Campaña"] == "Zurich")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Zurich.to_excel("F:/Reportes SF/Zurich/Zurich_Base.xlsx", index=False)

Amsa = reportes[
    (reportes["Campaña"] == "Zurich")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    & (reportes["Holding"] == "Antofagasta Minerals S.A.")
]
Amsa.to_excel("F:/Reportes SF/Zurich/Amsa.xlsx", index=False)

Bechtel = reportes[
    (reportes["Campaña"] == "Zurich")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    & (reportes["Holding"] == "Bechtel")
]
Bechtel.to_excel("F:/Reportes SF/Zurich/Bechtel.xlsx", index=False)

Pesco

In [35]:
Pesco = reportes[
    (reportes["Campaña"] == "Pesco")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    
]
Pesco.to_excel("F:/Reportes SF/Pesco/Pesco_Base.xlsx", index=False)

    


Milan Fajanovic

In [36]:
Milan = reportes[
    (reportes["Campaña"] == "Milan Fajanovic")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    
]
Milan.to_excel("F:/Reportes SF/Milan/Milan_Fajanovic_Base.xlsx", index=False)

Wom

In [37]:
wom = reportes[
    (reportes["Campaña"] == "MetLife")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    & (reportes["Empleador"] == "Wom S.A.")
]
wom.to_excel("F:/Reportes SF/Wom/Wom_Base.xlsx", index=False)

In [38]:
Forus = reportes[
    (reportes["Campaña"] == "Forus")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    
]
Forus.to_excel("F:/Reportes SF/Forus/Forus_Base.xlsx", index=False)

In [39]:
Cruz = reportes[
    (reportes["Campaña"] == "Farmacias Cruz Verde")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    
]
Cruz.to_excel("F:/Reportes SF/Cruz Verde/Cruz_Base.xlsx", index=False)
Cruz.shape

(1865, 22)

METLIFE

In [40]:
Metlife = reportes[
    (reportes["Campaña"] == "MetLife")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Metlife.to_excel("F:/Reportes SF/MetLife/Metlife_Base.xlsx", index=False)

Conosur = reportes[
    (reportes["Codigo EMP"] == "EMP5976")
    | (reportes["Codigo EMP"] == "EMP5977")
    | (reportes["Codigo EMP"] == "EMP5978")
    | (reportes["Codigo EMP"] == "EMP5979")
    | (reportes["Codigo EMP"] == "EMP5980")
    | (reportes["Codigo EMP"] == "EMP5981")
    | (reportes["Codigo EMP"] == "EMP5982")
    | (reportes["Codigo EMP"] == "EMP5983")
    | (reportes["Codigo EMP"] == "EMP5984")
    | (reportes["Codigo EMP"] == "EMP5985")
    | (reportes["Codigo EMP"] == "EMP5985")
    | (reportes["Codigo EMP"] == "EMP7431")
    | (reportes["Codigo EMP"] == "EMP2828")
]
Conosur.to_excel("F:/Reportes SF/MetLife/Conosur_Base.xlsx", index=False)

In [41]:
Burgos = reportes[
    
    (reportes["Codigo EMP"] == "EMP5924")
    | (reportes["Codigo EMP"] == "EMP5925")
    | (reportes["Codigo EMP"] == "EMP5926")
    | (reportes["Codigo EMP"] == "EMP5927")
    | (reportes["Codigo EMP"] == "EMP5928")
    | (reportes["Codigo EMP"] == "EMP5971")
    | (reportes["Codigo EMP"] == "EMP5972")
    | (reportes["Codigo EMP"] == "EMP5973")
    | (reportes["Codigo EMP"] == "EMP5974")
    | (reportes["Codigo EMP"] == "EMP5975")
    | (reportes["Codigo EMP"] == "EMP7259")
    | (reportes["Codigo EMP"] == "EMP6536")
    | (reportes["Codigo EMP"] == "EMP6959")
    | (reportes["Codigo EMP"] == "EMP6388")
     
]

Burgos = Burgos[(Burgos["Empleador"] == Burgos["Nombre Empresa"])]
Burgos.to_excel("F:/Reportes SF/MetLife/Burgos_Base.xlsx", index=False)

In [42]:
Ey = reportes[(reportes["Campaña"] == "EY") & (reportes["Empleador"] == reportes["Nombre Empresa"])]
Ey.to_excel("F:/Reportes SF/EY/EY_Base.xlsx", index=False)

In [43]:
Willis = reportes[
    
    (reportes["Codigo EMP"] == "EMP5924")
    | (reportes["Codigo EMP"] == "EMP5925")
    | (reportes["Codigo EMP"] == "EMP5926")
    | (reportes["Codigo EMP"] == "EMP5927")
    | (reportes["Codigo EMP"] == "EMP5928")
    | (reportes["Codigo EMP"] == "EMP5971")
    | (reportes["Codigo EMP"] == "EMP5972")
    | (reportes["Codigo EMP"] == "EMP5973")
    | (reportes["Codigo EMP"] == "EMP5974")
    | (reportes["Codigo EMP"] == "EMP5975")
    | (reportes["Codigo EMP"] == "EMP7259")
    | (reportes["Codigo EMP"] == "EMP6536")
    | (reportes["Codigo EMP"] == "EMP6959")
    | (reportes["Codigo EMP"] == "EMP6388")
    
    | (reportes["Codigo EMP"] == "EMP5694")
    | (reportes["Codigo EMP"] == "EMP5655")
    | (reportes["Codigo EMP"] == "EMP5681")
    | (reportes["Codigo EMP"] == "EMP5682")
    | (reportes["Codigo EMP"] == "EMP5684")
    | (reportes["Codigo EMP"] == "EMP6274")
    | (reportes["Codigo EMP"] == "EMP5685")
    | (reportes["Codigo EMP"] == "EMP5686")
    | (reportes["Codigo EMP"] == "EMP5687")
    | (reportes["Codigo EMP"] == "EMP5688")
    | (reportes["Codigo EMP"] == "EMP5689")
    | (reportes["Codigo EMP"] == "EMP5690")
    | (reportes["Codigo EMP"] == "EMP5691")

    | (reportes["Codigo EMP"] == "EMP5692")
    | (reportes["Codigo EMP"] == "EMP5693")
    | (reportes["Codigo EMP"] == "EMP5694")
    | (reportes["Codigo EMP"] == "EMP5695")
    | (reportes["Codigo EMP"] == "EMP5696")
    | (reportes["Codigo EMP"] == "EMP5697")
    | (reportes["Codigo EMP"] == "EMP5698")
    | (reportes["Codigo EMP"] == "EMP5639")
    | (reportes["Codigo EMP"] == "EMP5640")
    | (reportes["Codigo EMP"] == "EMP5641")
    | (reportes["Codigo EMP"] == "EMP5642")
    | (reportes["Codigo EMP"] == "EMP5643")
    | (reportes["Codigo EMP"] == "EMP5644")

    | (reportes["Codigo EMP"] == "EMP5645")
    | (reportes["Codigo EMP"] == "EMP5646")
    | (reportes["Codigo EMP"] == "EMP5647")
    | (reportes["Codigo EMP"] == "EMP5648")
    | (reportes["Codigo EMP"] == "EMP5649")
    | (reportes["Codigo EMP"] == "EMP5650")
    | (reportes["Codigo EMP"] == "EMP5651")
    | (reportes["Codigo EMP"] == "EMP5652")
    | (reportes["Codigo EMP"] == "EMP5653")
    | (reportes["Codigo EMP"] == "EMP5654")
    | (reportes["Codigo EMP"] == "EMP5655")
    | (reportes["Codigo EMP"] == "EMP7526")
    | (reportes["Codigo EMP"] == "EMP6760")

    | (reportes["Codigo EMP"] == "EMP7678")
    | (reportes["Codigo EMP"] == "EMP6888")
    | (reportes["Codigo EMP"] == "EMP6444")
    | (reportes["Codigo EMP"] == "EMP6406")
    | (reportes["Codigo EMP"] == "EMP5616")
    | (reportes["Codigo EMP"] == "EMP9007")
    
     
]

Willis = Willis[(Willis["Empleador"] == Willis["Nombre Empresa"])]
Willis.to_excel("F:/Reportes SF/MetLife/Willis_Base.xlsx", index=False)

In [44]:
Aon = reportes[
 (reportes["Codigo EMP"] == "EMP5627")
| (reportes["Codigo EMP"] == "EMP9181")
| (reportes["Codigo EMP"] == "EMP9183")
| (reportes["Codigo EMP"] == "EMP9184")
| (reportes["Codigo EMP"] == "EMP9185")
| (reportes["Codigo EMP"] == "EMP9186")
| (reportes["Codigo EMP"] == "EMP9241")
| (reportes["Codigo EMP"] == "EMP9256")
| (reportes["Codigo EMP"] == "EMP9187")
| (reportes["Codigo EMP"] == "EMP9188")
| (reportes["Codigo EMP"] == "EMP9189")
| (reportes["Codigo EMP"] == "EMP9230")
| (reportes["Codigo EMP"] == "EMP9359")
| (reportes["Codigo EMP"] == "EMP9365")
| (reportes["Codigo EMP"] == "EMP9366")
| (reportes["Codigo EMP"] == "EMP9367")
| (reportes["Codigo EMP"] == "EMP9368")
| (reportes["Codigo EMP"] == "EMP5890")
| (reportes["Codigo EMP"] == "EMP5891")
| (reportes["Codigo EMP"] == "EMP5892")
| (reportes["Codigo EMP"] == "EMP5893")
| (reportes["Codigo EMP"] == "EMP5894")
| (reportes["Codigo EMP"] == "EMP5895")
| (reportes["Codigo EMP"] == "EMP5896")
| (reportes["Codigo EMP"] == "EMP5897")
| (reportes["Codigo EMP"] == "EMP5898")
| (reportes["Codigo EMP"] == "EMP5899")
| (reportes["Codigo EMP"] == "EMP5930")
| (reportes["Codigo EMP"] == "EMP5931")
| (reportes["Codigo EMP"] == "EMP5932")
| (reportes["Codigo EMP"] == "EMP5933")
| (reportes["Codigo EMP"] == "EMP5934")
| (reportes["Codigo EMP"] == "EMP5935")
| (reportes["Codigo EMP"] == "EMP5936")
| (reportes["Codigo EMP"] == "EMP5937")
| (reportes["Codigo EMP"] == "EMP5939")
| (reportes["Codigo EMP"] == "EMP5940")
| (reportes["Codigo EMP"] == "EMP5941")
| (reportes["Codigo EMP"] == "EMP5942")
| (reportes["Codigo EMP"] == "EMP5943")
| (reportes["Codigo EMP"] == "EMP5944")
| (reportes["Codigo EMP"] == "EMP5945")
| (reportes["Codigo EMP"] == "EMP5946")
| (reportes["Codigo EMP"] == "EMP5947")
| (reportes["Codigo EMP"] == "EMP5949")
| (reportes["Codigo EMP"] == "EMP9436")
| (reportes["Codigo EMP"] == "EMP9437")
| (reportes["Codigo EMP"] == "EMP9861")
| (reportes["Codigo EMP"] == "EMP9862")
| (reportes["Codigo EMP"] == "EMP9864")
| (reportes["Codigo EMP"] == "EMP9865")
| (reportes["Codigo EMP"] == "EMP9866")
| (reportes["Codigo EMP"] == "EMP9873")
| (reportes["Codigo EMP"] == "EMP9874")
| (reportes["Codigo EMP"] == "EMP9875")
| (reportes["Codigo EMP"] == "EMP9892")
| (reportes["Codigo EMP"] == "EMP9893")
| (reportes["Codigo EMP"] == "EMP9899")
| (reportes["Codigo EMP"] == "EMP9901")
| (reportes["Codigo EMP"] == "EMP9902")
| (reportes["Codigo EMP"] == "EMP9903")
| (reportes["Codigo EMP"] == "EMP9904")
| (reportes["Codigo EMP"] == "EMP9905")
| (reportes["Codigo EMP"] == "EMP9906")
| (reportes["Codigo EMP"] == "EMP9907")
| (reportes["Codigo EMP"] == "EMP9908")
| (reportes["Codigo EMP"] == "EMP9909")
| (reportes["Codigo EMP"] == "EMP9493")
| (reportes["Codigo EMP"] == "EMP9494")
| (reportes["Codigo EMP"] == "EMP9518")
| (reportes["Codigo EMP"] == "EMP9539")
| (reportes["Codigo EMP"] == "EMP9540")
| (reportes["Codigo EMP"] == "EMP9558")
| (reportes["Codigo EMP"] == "EMP9741")
| (reportes["Codigo EMP"] == "EMP9742")
| (reportes["Codigo EMP"] == "EMP9743")
| (reportes["Codigo EMP"] == "EMP9745")
| (reportes["Codigo EMP"] == "EMP9747")
| (reportes["Codigo EMP"] == "EMP9630")
| (reportes["Codigo EMP"] == "EMP9631")
| (reportes["Codigo EMP"] == "EMP9632")
| (reportes["Codigo EMP"] == "EMP9638")
| (reportes["Codigo EMP"] == "EMP9755")
| (reportes["Codigo EMP"] == "EMP6532")
| (reportes["Codigo EMP"] == "EMP6538")
| (reportes["Codigo EMP"] == "EMP6555")
| (reportes["Codigo EMP"] == "EMP6556")
| (reportes["Codigo EMP"] == "EMP6557")
| (reportes["Codigo EMP"] == "EMP6558")
| (reportes["Codigo EMP"] == "EMP6560")
| (reportes["Codigo EMP"] == "EMP6561")
| (reportes["Codigo EMP"] == "EMP6562")
| (reportes["Codigo EMP"] == "EMP6563")
| (reportes["Codigo EMP"] == "EMP6671")
| (reportes["Codigo EMP"] == "EMP6717")
| (reportes["Codigo EMP"] == "EMP6720")
| (reportes["Codigo EMP"] == "EMP6786")
| (reportes["Codigo EMP"] == "EMP6864")
| (reportes["Codigo EMP"] == "EMP6865")
| (reportes["Codigo EMP"] == "EMP6866")
| (reportes["Codigo EMP"] == "EMP6867")
| (reportes["Codigo EMP"] == "EMP6868")
| (reportes["Codigo EMP"] == "EMP6938")
| (reportes["Codigo EMP"] == "EMP7003")
| (reportes["Codigo EMP"] == "EMP7004")
| (reportes["Codigo EMP"] == "EMP7083")
| (reportes["Codigo EMP"] == "EMP7176")
| (reportes["Codigo EMP"] == "EMP7177")
| (reportes["Codigo EMP"] == "EMP7269")
| (reportes["Codigo EMP"] == "EMP7337")
| (reportes["Codigo EMP"] == "EMP7396")
| (reportes["Codigo EMP"] == "EMP7418")
| (reportes["Codigo EMP"] == "EMP7513")
| (reportes["Codigo EMP"] == "EMP7646")
| (reportes["Codigo EMP"] == "EMP7808")
| (reportes["Codigo EMP"] == "EMP7863")
| (reportes["Codigo EMP"] == "EMP9183")
| (reportes["Codigo EMP"] == "EMP9184")
| (reportes["Codigo EMP"] == "EMP9185")
| (reportes["Codigo EMP"] == "EMP9186")
| (reportes["Codigo EMP"] == "EMP9256")
| (reportes["Codigo EMP"] == "EMP9187")
| (reportes["Codigo EMP"] == "EMP9188")
| (reportes["Codigo EMP"] == "EMP9189")
| (reportes["Codigo EMP"] == "EMP5890")
| (reportes["Codigo EMP"] == "EMP5891")
| (reportes["Codigo EMP"] == "EMP5892")
| (reportes["Codigo EMP"] == "EMP5893")
| (reportes["Codigo EMP"] == "EMP5894")
| (reportes["Codigo EMP"] == "EMP5895")
| (reportes["Codigo EMP"] == "EMP5896")
| (reportes["Codigo EMP"] == "EMP5897")
| (reportes["Codigo EMP"] == "EMP5898")
| (reportes["Codigo EMP"] == "EMP5899")
| (reportes["Codigo EMP"] == "EMP5930")
| (reportes["Codigo EMP"] == "EMP5931")
| (reportes["Codigo EMP"] == "EMP5932")
| (reportes["Codigo EMP"] == "EMP5933")
| (reportes["Codigo EMP"] == "EMP5934")
| (reportes["Codigo EMP"] == "EMP5935")
| (reportes["Codigo EMP"] == "EMP5936")
| (reportes["Codigo EMP"] == "EMP5937")
| (reportes["Codigo EMP"] == "EMP5939")
| (reportes["Codigo EMP"] == "EMP5940")
| (reportes["Codigo EMP"] == "EMP5941")
| (reportes["Codigo EMP"] == "EMP5944")
| (reportes["Codigo EMP"] == "EMP5947")
| (reportes["Codigo EMP"] == "EMP5949")
| (reportes["Codigo EMP"] == "EMP9436")
| (reportes["Codigo EMP"] == "EMP9863")
| (reportes["Codigo EMP"] == "EMP9864")
| (reportes["Codigo EMP"] == "EMP9865")
| (reportes["Codigo EMP"] == "EMP9866")
| (reportes["Codigo EMP"] == "EMP9901")
| (reportes["Codigo EMP"] == "EMP9902")
| (reportes["Codigo EMP"] == "EMP9903")
| (reportes["Codigo EMP"] == "EMP9904")
| (reportes["Codigo EMP"] == "EMP9905")
| (reportes["Codigo EMP"] == "EMP9906")
| (reportes["Codigo EMP"] == "EMP9907")
| (reportes["Codigo EMP"] == "EMP9908")
| (reportes["Codigo EMP"] == "EMP9909")
| (reportes["Codigo EMP"] == "EMP9493")
| (reportes["Codigo EMP"] == "EMP9494")
| (reportes["Codigo EMP"] == "EMP9539")
| (reportes["Codigo EMP"] == "EMP9540")
| (reportes["Codigo EMP"] == "EMP9558")
| (reportes["Codigo EMP"] == "EMP9741")
| (reportes["Codigo EMP"] == "EMP9742")
| (reportes["Codigo EMP"] == "EMP9627")
| (reportes["Codigo EMP"] == "EMP9630")
| (reportes["Codigo EMP"] == "EMP9631")
| (reportes["Codigo EMP"] == "EMP9632")
| (reportes["Codigo EMP"] == "EMP9638")
| (reportes["Codigo EMP"] == "EMP6786")
| (reportes["Codigo EMP"] == "EMP7597")

]
Aon = Aon[(Aon["Empleador"] == Aon["Nombre Empresa"])]
Aon.to_excel("F:/Reportes SF/MetLife/Aon_Base.xlsx", index=False)

In [45]:
Aramark = reportes[
 (reportes["Codigo EMP"] == "EMP9244")
| (reportes["Codigo EMP"] == "EMP9245")
| (reportes["Codigo EMP"] == "EMP9246")
| (reportes["Codigo EMP"] == "EMP9247")
| (reportes["Codigo EMP"] == "EMP9248")
| (reportes["Codigo EMP"] == "EMP9249")
| (reportes["Codigo EMP"] == "EMP9250")
| (reportes["Codigo EMP"] == "EMP9148")
| (reportes["Codigo EMP"] == "EMP9149")
| (reportes["Codigo EMP"] == "EMP9150")
| (reportes["Codigo EMP"] == "EMP9151")
| (reportes["Codigo EMP"] == "EMP9152")
| (reportes["Codigo EMP"] == "EMP9153")
| (reportes["Codigo EMP"] == "EMP9154")
| (reportes["Codigo EMP"] == "EMP9155")
| (reportes["Codigo EMP"] == "EMP9156")
| (reportes["Codigo EMP"] == "EMP9157")
| (reportes["Codigo EMP"] == "EMP9158")
| (reportes["Codigo EMP"] == "EMP9159")
| (reportes["Codigo EMP"] == "EMP9160")
| (reportes["Codigo EMP"] == "EMP9161")
| (reportes["Codigo EMP"] == "EMP9162")
| (reportes["Codigo EMP"] == "EMP9163")
| (reportes["Codigo EMP"] == "EMP9164")
| (reportes["Codigo EMP"] == "EMP9165")
| (reportes["Codigo EMP"] == "EMP9166")
| (reportes["Codigo EMP"] == "EMP9167")
| (reportes["Codigo EMP"] == "EMP5806")
| (reportes["Codigo EMP"] == "EMP5808")
| (reportes["Codigo EMP"] == "EMP5810")
| (reportes["Codigo EMP"] == "EMP9266")
| (reportes["Codigo EMP"] == "EMP9267")
| (reportes["Codigo EMP"] == "EMP9268")
| (reportes["Codigo EMP"] == "EMP9269")
| (reportes["Codigo EMP"] == "EMP9270")
| (reportes["Codigo EMP"] == "EMP9271")

]
Aramark = Aramark[(Aramark["Empleador"] == Aramark["Nombre Empresa"])]
Aramark.to_excel("F:/Reportes SF/MetLife/Aramark_Base.xlsx", index=False)

In [46]:
Benefits = reportes[
    (reportes["Codigo EMP"] == "EMP9190")
    | (reportes["Codigo EMP"] == "EMP9191")
    | (reportes["Codigo EMP"] == "EMP9192")
    | (reportes["Codigo EMP"] == "EMP9193")
    | (reportes["Codigo EMP"] == "EMP9194")
    | (reportes["Codigo EMP"] == "EMP9195")
    | (reportes["Codigo EMP"] == "EMP9196")
    | (reportes["Codigo EMP"] == "EMP9197")
    | (reportes["Codigo EMP"] == "EMP9198")
    | (reportes["Codigo EMP"] == "EMP9199")
    | (reportes["Codigo EMP"] == "EMP9200")
    | (reportes["Codigo EMP"] == "EMP9201")
    | (reportes["Codigo EMP"] == "EMP9202")
    | (reportes["Codigo EMP"] == "EMP9203")
    | (reportes["Codigo EMP"] == "EMP9204")
    | (reportes["Codigo EMP"] == "EMP9205")
    | (reportes["Codigo EMP"] == "EMP9326")
    | (reportes["Codigo EMP"] == "EMP9327")
    | (reportes["Codigo EMP"] == "EMP9328")
    | (reportes["Codigo EMP"] == "EMP9340")
    | (reportes["Codigo EMP"] == "EMP9345")
    | (reportes["Codigo EMP"] == "EMP9349")
    | (reportes["Codigo EMP"] == "EMP9324")
    | (reportes["Codigo EMP"] == "EMP5908")
    | (reportes["Codigo EMP"] == "EMP5909")
    | (reportes["Codigo EMP"] == "EMP5912")
    | (reportes["Codigo EMP"] == "EMP5913")
    | (reportes["Codigo EMP"] == "EMP5914")
    | (reportes["Codigo EMP"] == "EMP5915")
    | (reportes["Codigo EMP"] == "EMP5916")
    | (reportes["Codigo EMP"] == "EMP5917")
    | (reportes["Codigo EMP"] == "EMP5919")
    | (reportes["Codigo EMP"] == "EMP5920")
    | (reportes["Codigo EMP"] == "EMP5921")
    | (reportes["Codigo EMP"] == "EMP5922")
    | (reportes["Codigo EMP"] == "EMP5923")
    | (reportes["Codigo EMP"] == "EMP6001")
    | (reportes["Codigo EMP"] == "EMP9414")
    | (reportes["Codigo EMP"] == "EMP9464")
    | (reportes["Codigo EMP"] == "EMP9840")
    | (reportes["Codigo EMP"] == "EMP9852")
    | (reportes["Codigo EMP"] == "EMP9703")
    | (reportes["Codigo EMP"] == "EMP9600")
    | (reportes["Codigo EMP"] == "EMP9601")
    | (reportes["Codigo EMP"] == "EMP9602")
    | (reportes["Codigo EMP"] == "EMP9603")
    | (reportes["Codigo EMP"] == "EMP9604")
    | (reportes["Codigo EMP"] == "EMP9605")
    | (reportes["Codigo EMP"] == "EMP9606")
    | (reportes["Codigo EMP"] == "EMP9608")
    | (reportes["Codigo EMP"] == "EMP9609")
    | (reportes["Codigo EMP"] == "EMP9610")
    | (reportes["Codigo EMP"] == "EMP9611")
    | (reportes["Codigo EMP"] == "EMP9612")
    | (reportes["Codigo EMP"] == "EMP9613")
    | (reportes["Codigo EMP"] == "EMP9614")
    | (reportes["Codigo EMP"] == "EMP9615")
    | (reportes["Codigo EMP"] == "EMP9616")
    | (reportes["Codigo EMP"] == "EMP9617")
    | (reportes["Codigo EMP"] == "EMP9618")
    | (reportes["Codigo EMP"] == "EMP9619")
    | (reportes["Codigo EMP"] == "EMP9784")
    | (reportes["Codigo EMP"] == "EMP9670")
    | (reportes["Codigo EMP"] == "EMP9674")
    | (reportes["Codigo EMP"] == "EMP9802")
    | (reportes["Codigo EMP"] == "EMP6534")
    | (reportes["Codigo EMP"] == "EMP6540")
    | (reportes["Codigo EMP"] == "EMP6564")
    | (reportes["Codigo EMP"] == "EMP6621")
    | (reportes["Codigo EMP"] == "EMP6640")
    | (reportes["Codigo EMP"] == "EMP6643")
    | (reportes["Codigo EMP"] == "EMP6650")
    | (reportes["Codigo EMP"] == "EMP6653")
    | (reportes["Codigo EMP"] == "EMP6661")
    | (reportes["Codigo EMP"] == "EMP6758")
    | (reportes["Codigo EMP"] == "EMP6763")
    | (reportes["Codigo EMP"] == "EMP6772")
    | (reportes["Codigo EMP"] == "EMP6783")
    | (reportes["Codigo EMP"] == "EMP6792")
    | (reportes["Codigo EMP"] == "EMP6829")
    | (reportes["Codigo EMP"] == "EMP6869")
    | (reportes["Codigo EMP"] == "EMP6935")
    | (reportes["Codigo EMP"] == "EMP6940")
    | (reportes["Codigo EMP"] == "EMP7005")
    | (reportes["Codigo EMP"] == "EMP7033")
    | (reportes["Codigo EMP"] == "EMP7072")
    | (reportes["Codigo EMP"] == "EMP7189")
    | (reportes["Codigo EMP"] == "EMP7243")
    | (reportes["Codigo EMP"] == "EMP7247")
    | (reportes["Codigo EMP"] == "EMP7345")
    | (reportes["Codigo EMP"] == "EMP7507")
    | (reportes["Codigo EMP"] == "EMP7508")
    | (reportes["Codigo EMP"] == "EMP7546")
    | (reportes["Codigo EMP"] == "EMP7564")
    | (reportes["Codigo EMP"] == "EMP7567")
    | (reportes["Codigo EMP"] == "EMP7617")
    | (reportes["Codigo EMP"] == "EMP7639")
    | (reportes["Codigo EMP"] == "EMP7663")
    | (reportes["Codigo EMP"] == "EMP7674")
    | (reportes["Codigo EMP"] == "EMP7702")
    | (reportes["Codigo EMP"] == "EMP7739")
    | (reportes["Codigo EMP"] == "EMP7744")
    | (reportes["Codigo EMP"] == "EMP7784")
    | (reportes["Codigo EMP"] == "EMP7785")
    | (reportes["Codigo EMP"] == "EMP9206")
    | (reportes["Codigo EMP"] == "EMP5910")
    | (reportes["Codigo EMP"] == "EMP5911")
    | (reportes["Codigo EMP"] == "EMP5918")
    | (reportes["Codigo EMP"] == "EMP9607")
]
Benefits = Benefits[(Benefits["Empleador"] == Benefits["Nombre Empresa"])]
Benefits.to_excel("F:/Reportes SF/MetLife/Benefits_Base.xlsx", index=False)

In [47]:
Interno = reportes[
    (reportes["Codigo EMP"] == "EMP5999")
    | (reportes["Codigo EMP"] == "EMP9683")
    | (reportes["Codigo EMP"] == "EMP9684")
    | (reportes["Codigo EMP"] == "EMP9685")
    | (reportes["Codigo EMP"] == "EMP9686")
    | (reportes["Codigo EMP"] == "EMP9687")
    | (reportes["Codigo EMP"] == "EMP9688")
    | (reportes["Codigo EMP"] == "EMP9689")
    | (reportes["Codigo EMP"] == "EMP9690")
    | (reportes["Codigo EMP"] == "EMP9691")
    | (reportes["Codigo EMP"] == "EMP7881")
]
Interno = Interno[(Interno["Empleador"] == Interno["Nombre Empresa"])]
Interno.to_excel("F:/Reportes SF/MetLife/Interno_Base.xlsx", index=False)

In [48]:
Lorena = reportes[
    (reportes["Codigo EMP"] == "EMP5786")
    | (reportes["Codigo EMP"] == "EMP9171")
    | (reportes["Codigo EMP"] == "EMP5785")
    | (reportes["Codigo EMP"] == "EMP5787")
    | (reportes["Codigo EMP"] == "EMP5788")
    | (reportes["Codigo EMP"] == "EMP5789")
    | (reportes["Codigo EMP"] == "EMP5790")
    | (reportes["Codigo EMP"] == "EMP5791")
    | (reportes["Codigo EMP"] == "EMP5792")
    | (reportes["Codigo EMP"] == "EMP5793")
    | (reportes["Codigo EMP"] == "EMP5794")
    | (reportes["Codigo EMP"] == "EMP9310")
    | (reportes["Codigo EMP"] == "EMP9441")
    | (reportes["Codigo EMP"] == "EMP9792")
    | (reportes["Codigo EMP"] == "EMP6830")
    | (reportes["Codigo EMP"] == "EMP6975")
    | (reportes["Codigo EMP"] == "EMP7014")
    | (reportes["Codigo EMP"] == "EMP7045")
    | (reportes["Codigo EMP"] == "EMP7060")
    | (reportes["Codigo EMP"] == "EMP7422")
]
Lorena = Lorena[(Lorena["Empleador"] == Lorena["Nombre Empresa"])]
Lorena.to_excel("F:/Reportes SF/MetLife/Lorena_Base.xlsx", index=False)

In [49]:
Security_metlife = reportes[
 (reportes["Codigo EMP"] == "EMP5952")
| (reportes["Codigo EMP"] == "EMP5957")
| (reportes["Codigo EMP"] == "EMP5966")
| (reportes["Codigo EMP"] == "EMP5989")
| (reportes["Codigo EMP"] == "EMP6833")
| (reportes["Codigo EMP"] == "EMP6902")
| (reportes["Codigo EMP"] == "EMP7332")
| (reportes["Codigo EMP"] == "EMP7661")
| (reportes["Codigo EMP"] == "EMP9232")
| (reportes["Codigo EMP"] == "EMP9233")
| (reportes["Codigo EMP"] == "EMP9234")
| (reportes["Codigo EMP"] == "EMP9235")
| (reportes["Codigo EMP"] == "EMP9236")
| (reportes["Codigo EMP"] == "EMP9237")
| (reportes["Codigo EMP"] == "EMP9238")
| (reportes["Codigo EMP"] == "EMP9239")
| (reportes["Codigo EMP"] == "EMP9240")
| (reportes["Codigo EMP"] == "EMP9347")
| (reportes["Codigo EMP"] == "EMP9287")
| (reportes["Codigo EMP"] == "EMP5950")
| (reportes["Codigo EMP"] == "EMP5951")
| (reportes["Codigo EMP"] == "EMP5953")
| (reportes["Codigo EMP"] == "EMP5955")
| (reportes["Codigo EMP"] == "EMP5956")
| (reportes["Codigo EMP"] == "EMP5958")
| (reportes["Codigo EMP"] == "EMP5959")
| (reportes["Codigo EMP"] == "EMP5960")
| (reportes["Codigo EMP"] == "EMP5961")
| (reportes["Codigo EMP"] == "EMP5962")
| (reportes["Codigo EMP"] == "EMP5963")
| (reportes["Codigo EMP"] == "EMP5964")
| (reportes["Codigo EMP"] == "EMP5965")
| (reportes["Codigo EMP"] == "EMP5967")
| (reportes["Codigo EMP"] == "EMP5968")
| (reportes["Codigo EMP"] == "EMP5969")
| (reportes["Codigo EMP"] == "EMP5990")
| (reportes["Codigo EMP"] == "EMP5991")
| (reportes["Codigo EMP"] == "EMP9440")
| (reportes["Codigo EMP"] == "EMP9454")
| (reportes["Codigo EMP"] == "EMP9455")
| (reportes["Codigo EMP"] == "EMP9456")
| (reportes["Codigo EMP"] == "EMP9457")
| (reportes["Codigo EMP"] == "EMP9458")
| (reportes["Codigo EMP"] == "EMP9459")
| (reportes["Codigo EMP"] == "EMP9460")
| (reportes["Codigo EMP"] == "EMP9512")
| (reportes["Codigo EMP"] == "EMP9553")
| (reportes["Codigo EMP"] == "EMP9669")
| (reportes["Codigo EMP"] == "EMP6434")
| (reportes["Codigo EMP"] == "EMP6713")
| (reportes["Codigo EMP"] == "EMP6877")
| (reportes["Codigo EMP"] == "EMP6898")
| (reportes["Codigo EMP"] == "EMP6901")
| (reportes["Codigo EMP"] == "EMP6905")
| (reportes["Codigo EMP"] == "EMP7010")
| (reportes["Codigo EMP"] == "EMP7066")
| (reportes["Codigo EMP"] == "EMP7080")
| (reportes["Codigo EMP"] == "EMP7094")
| (reportes["Codigo EMP"] == "EMP7150")
| (reportes["Codigo EMP"] == "EMP7333")
| (reportes["Codigo EMP"] == "EMP7504")
| (reportes["Codigo EMP"] == "EMP7505")
| (reportes["Codigo EMP"] == "EMP7623")
| (reportes["Codigo EMP"] == "EMP7625")
| (reportes["Codigo EMP"] == "EMP7651")
| (reportes["Codigo EMP"] == "EMP7721")
| (reportes["Codigo EMP"] == "EMP7724")

]
Security_metlife = Security_metlife[(Security_metlife["Empleador"] == Security_metlife["Nombre Empresa"])]
Security_metlife.to_excel("F:/Reportes SF/MetLife/Security_Base.xlsx", index=False)

In [50]:
Thb = reportes[
    (reportes["Codigo EMP"] == "EMP9336")
    | (reportes["Codigo EMP"] == "EMP9442")
    | (reportes["Codigo EMP"] == "EMP9746")
    | (reportes["Codigo EMP"] == "EMP9627")
    | (reportes["Codigo EMP"] == "EMP5618")
    | (reportes["Codigo EMP"] == "EMP5619")
    | (reportes["Codigo EMP"] == "EMP5620")
    | (reportes["Codigo EMP"] == "EMP5621")
    | (reportes["Codigo EMP"] == "EMP5622")
    | (reportes["Codigo EMP"] == "EMP5623")
    | (reportes["Codigo EMP"] == "EMP5624")
    | (reportes["Codigo EMP"] == "EMP5625")
    | (reportes["Codigo EMP"] == "EMP5626")
    | (reportes["Codigo EMP"] == "EMP5628")
    | (reportes["Codigo EMP"] == "EMP5629")
    | (reportes["Codigo EMP"] == "EMP5630")
    | (reportes["Codigo EMP"] == "EMP5631")
    | (reportes["Codigo EMP"] == "EMP5632")
    | (reportes["Codigo EMP"] == "EMP5633")
    | (reportes["Codigo EMP"] == "EMP5634")
    | (reportes["Codigo EMP"] == "EMP5635")
    | (reportes["Codigo EMP"] == "EMP5636")
    | (reportes["Codigo EMP"] == "EMP5637")
    | (reportes["Codigo EMP"] == "EMP5638")
    | (reportes["Codigo EMP"] == "EMP5669")
    | (reportes["Codigo EMP"] == "EMP5670")
    | (reportes["Codigo EMP"] == "EMP5671")
    | (reportes["Codigo EMP"] == "EMP5672")
    | (reportes["Codigo EMP"] == "EMP5673")
    | (reportes["Codigo EMP"] == "EMP5674")
    | (reportes["Codigo EMP"] == "EMP5675")
    | (reportes["Codigo EMP"] == "EMP5677")
    | (reportes["Codigo EMP"] == "EMP5721")
    | (reportes["Codigo EMP"] == "EMP5722")
    | (reportes["Codigo EMP"] == "EMP5723")
    | (reportes["Codigo EMP"] == "EMP5724")
    | (reportes["Codigo EMP"] == "EMP5725")
    | (reportes["Codigo EMP"] == "EMP5726")
    | (reportes["Codigo EMP"] == "EMP5727")
    | (reportes["Codigo EMP"] == "EMP5728")
    | (reportes["Codigo EMP"] == "EMP5729")
    | (reportes["Codigo EMP"] == "EMP5730")
    | (reportes["Codigo EMP"] == "EMP5731")
    | (reportes["Codigo EMP"] == "EMP5732")
    | (reportes["Codigo EMP"] == "EMP5733")
    | (reportes["Codigo EMP"] == "EMP5734")
    | (reportes["Codigo EMP"] == "EMP5735")
    | (reportes["Codigo EMP"] == "EMP5736")
    | (reportes["Codigo EMP"] == "EMP5737")
    | (reportes["Codigo EMP"] == "EMP5738")
    | (reportes["Codigo EMP"] == "EMP9242")
    | (reportes["Codigo EMP"] == "EMP9121")
    | (reportes["Codigo EMP"] == "EMP9122")
    | (reportes["Codigo EMP"] == "EMP9123")
    | (reportes["Codigo EMP"] == "EMP9168")
    | (reportes["Codigo EMP"] == "EMP9137")
    | (reportes["Codigo EMP"] == "EMP9320")
    | (reportes["Codigo EMP"] == "EMP9364")
    | (reportes["Codigo EMP"] == "EMP9257")
    | (reportes["Codigo EMP"] == "EMP9258")
    | (reportes["Codigo EMP"] == "EMP9259")
    | (reportes["Codigo EMP"] == "EMP9260")
    | (reportes["Codigo EMP"] == "EMP9261")
    | (reportes["Codigo EMP"] == "EMP9262")
    | (reportes["Codigo EMP"] == "EMP9461")
    | (reportes["Codigo EMP"] == "EMP9463")
    | (reportes["Codigo EMP"] == "EMP9859")
    | (reportes["Codigo EMP"] == "EMP9887")
    | (reportes["Codigo EMP"] == "EMP9888")
    | (reportes["Codigo EMP"] == "EMP9925")
    | (reportes["Codigo EMP"] == "EMP9952")
    | (reportes["Codigo EMP"] == "EMP9953")
    | (reportes["Codigo EMP"] == "EMP9514")
    | (reportes["Codigo EMP"] == "EMP9515")
    | (reportes["Codigo EMP"] == "EMP9516")
    | (reportes["Codigo EMP"] == "EMP9517")
    | (reportes["Codigo EMP"] == "EMP9956")
    | (reportes["Codigo EMP"] == "EMP9957")
    | (reportes["Codigo EMP"] == "EMP9958")
    | (reportes["Codigo EMP"] == "EMP9970")
    | (reportes["Codigo EMP"] == "EMP9559")
    | (reportes["Codigo EMP"] == "EMP9561")
    | (reportes["Codigo EMP"] == "EMP9565")
    | (reportes["Codigo EMP"] == "EMP9567")
    | (reportes["Codigo EMP"] == "EMP9568")
    | (reportes["Codigo EMP"] == "EMP9569")
    | (reportes["Codigo EMP"] == "EMP9570")
    | (reportes["Codigo EMP"] == "EMP9571")
    | (reportes["Codigo EMP"] == "EMP9572")
    | (reportes["Codigo EMP"] == "EMP9573")
    | (reportes["Codigo EMP"] == "EMP9574")
    | (reportes["Codigo EMP"] == "EMP9576")
    | (reportes["Codigo EMP"] == "EMP9698")
    | (reportes["Codigo EMP"] == "EMP9699")
    | (reportes["Codigo EMP"] == "EMP9700")
    | (reportes["Codigo EMP"] == "EMP9577")
    | (reportes["Codigo EMP"] == "EMP9580")
    | (reportes["Codigo EMP"] == "EMP9581")
    | (reportes["Codigo EMP"] == "EMP9583")
    | (reportes["Codigo EMP"] == "EMP9584")
    | (reportes["Codigo EMP"] == "EMP9585")
    | (reportes["Codigo EMP"] == "EMP9586")
    | (reportes["Codigo EMP"] == "EMP9587")
    | (reportes["Codigo EMP"] == "EMP9588")
    | (reportes["Codigo EMP"] == "EMP9594")
    | (reportes["Codigo EMP"] == "EMP9624")
    | (reportes["Codigo EMP"] == "EMP9625")
    | (reportes["Codigo EMP"] == "EMP9626")
    | (reportes["Codigo EMP"] == "EMP9628")
    | (reportes["Codigo EMP"] == "EMP9629")
    | (reportes["Codigo EMP"] == "EMP9633")
    | (reportes["Codigo EMP"] == "EMP9634")
    | (reportes["Codigo EMP"] == "EMP9635")
    | (reportes["Codigo EMP"] == "EMP9645")
    | (reportes["Codigo EMP"] == "EMP9791")
    | (reportes["Codigo EMP"] == "EMP9799")
    | (reportes["Codigo EMP"] == "EMP9806")
    | (reportes["Codigo EMP"] == "EMP9807")
    | (reportes["Codigo EMP"] == "EMP9691")
    | (reportes["Codigo EMP"] == "EMP6373")
    | (reportes["Codigo EMP"] == "EMP6397")
    | (reportes["Codigo EMP"] == "EMP6410")
    | (reportes["Codigo EMP"] == "EMP6481")
    | (reportes["Codigo EMP"] == "EMP6528")
    | (reportes["Codigo EMP"] == "EMP6612")
    | (reportes["Codigo EMP"] == "EMP6613")
    | (reportes["Codigo EMP"] == "EMP6753")
    | (reportes["Codigo EMP"] == "EMP6875")
    | (reportes["Codigo EMP"] == "EMP6891")
    | (reportes["Codigo EMP"] == "EMP7011")
    | (reportes["Codigo EMP"] == "EMP7012")
    | (reportes["Codigo EMP"] == "EMP7039")
    | (reportes["Codigo EMP"] == "EMP7040")
    | (reportes["Codigo EMP"] == "EMP7041")
    | (reportes["Codigo EMP"] == "EMP7051")
    | (reportes["Codigo EMP"] == "EMP7143")
    | (reportes["Codigo EMP"] == "EMP7170")
    | (reportes["Codigo EMP"] == "EMP7339")
    | (reportes["Codigo EMP"] == "EMP7420")
    | (reportes["Codigo EMP"] == "EMP7568")
    | (reportes["Codigo EMP"] == "EMP7626")
    | (reportes["Codigo EMP"] == "EMP7627")
    | (reportes["Codigo EMP"] == "EMP7633")
    | (reportes["Codigo EMP"] == "EMP7701")
    | (reportes["Codigo EMP"] == "EMP7740")
    | (reportes["Codigo EMP"] == "EMP7746")
    | (reportes["Codigo EMP"] == "EMP7772")
    | (reportes["Codigo EMP"] == "EMP7776")
]

Thb = Thb[(Thb["Empleador"] == Thb["Nombre Empresa"])]
Thb.to_excel("F:/Reportes SF/MetLife/Thb_Base.xlsx", index=False)

In [51]:
Veritas = reportes[
(reportes["Codigo EMP"] == "EMP5717")
| (reportes["Codigo EMP"] == "EMP5718")
| (reportes["Codigo EMP"] == "EMP5719")
| (reportes["Codigo EMP"] == "EMP5720")
| (reportes["Codigo EMP"] == "EMP9959")
| (reportes["Codigo EMP"] == "EMP9707")
| (reportes["Codigo EMP"] == "EMP9823")
| (reportes["Codigo EMP"] == "EMP9824")
| (reportes["Codigo EMP"] == "EMP6615")
| (reportes["Codigo EMP"] == "EMP6662")
| (reportes["Codigo EMP"] == "EMP6778")
| (reportes["Codigo EMP"] == "EMP6872")
| (reportes["Codigo EMP"] == "EMP7042")
| (reportes["Codigo EMP"] == "EMP7052")

]

Veritas = Veritas[(Veritas["Empleador"] == Veritas["Nombre Empresa"])]
Veritas.to_excel("F:/Reportes SF/MetLife/Veritas_Base.xlsx", index=False)

In [52]:
Mercado_Libre = reportes[
        (reportes["Codigo EMP"] == "EMP5617")
    ]

Mercado_Libre = Mercado_Libre[(Mercado_Libre["Empleador"] == Mercado_Libre["Nombre Empresa"])]
Mercado_Libre.to_excel("F:/Reportes SF/Mercado Libre/Mercado_Libre_Base.xlsx", index=False)


In [53]:
Ahumada = reportes[
        (reportes["Codigo EMP"] == "EMP6475")
    ]

Ahumada = Ahumada[(Ahumada["Empleador"] == Ahumada["Nombre Empresa"])]
Ahumada.to_excel("F:/Reportes SF/Farmacias Ahumada/Ahumada_Base.xlsx", index=False)


In [54]:
Minera_Abra = reportes[
        (reportes["Codigo EMP"] == "EMP7397")
        | (reportes["Codigo EMP"] == "EMP7795")
        |(reportes["Codigo EMP"] == "EMP7796")
    ]

Minera_Abra = Minera_Abra[(Minera_Abra["Empleador"] == Minera_Abra["Nombre Empresa"])]
Minera_Abra.to_excel("F:/Reportes SF/Minera El Abra/Abra_Base.xlsx", index=False)

In [55]:
Agrosuper = reportes[
        (reportes["Holding"] == "Agrosuper")
    ]

Agrosuper = Agrosuper[(Agrosuper["Empleador"] == Agrosuper["Nombre Empresa"])]
Agrosuper.to_excel("F:/Reportes SF/Metlife/Mercer_Agrosuper_Base.xlsx", index=False)

In [56]:
Lipigas = reportes[
        (reportes["Codigo EMP"] == "EMP7310")
    ]

Lipigas = Lipigas[(Lipigas["Empleador"] == Lipigas["Nombre Empresa"])]
Lipigas.to_excel("F:/Reportes SF/Lipigas/Lipigas_Base.xlsx", index=False)

In [57]:
Socofar = reportes[
    (reportes["Codigo EMP"] == "EMP3780")
    | (reportes["Codigo EMP"] == "EMP6749")
    | (reportes["Codigo EMP"] == "EMP6750")
    | (reportes["Codigo EMP"] == "EMP6751")
    | (reportes["Codigo EMP"] == "EMP7405")
    | (reportes["Codigo EMP"] == "EMP7406")
    | (reportes["Codigo EMP"] == "EMP7407")
]


Socofar = Socofar[(Socofar["Empleador"] == Socofar["Nombre Empresa"])]
Socofar.to_excel("F:/Reportes SF/Socofar/Socofar_Base.xlsx", index=False)

In [58]:
Kaufmann = reportes[
    (reportes["Codigo EMP"] == "EMP4171")
    | (reportes["Codigo EMP"] == "EMP7308")
    | (reportes["Codigo EMP"] == "EMP7309")
]


Kaufmann = Kaufmann[(Kaufmann["Empleador"] == Kaufmann["Nombre Empresa"])]
Kaufmann.to_excel("F:/Reportes SF/Kaufmann/Kaufmann_Base.xlsx", index=False)

In [59]:

Tressp = reportes[
    (reportes["Codigo EMP"] == "EMP6467")
    #| (reportes["Codigo EMP"] == "EMP6468")
    #| (reportes["Codigo EMP"] == "EMP7607")
]


Tressp = Tressp[(Tressp["Empleador"] == Tressp["Nombre Empresa"])]
Tressp.to_excel("F:/Reportes SF/3SP/3sp_Base.xlsx", index=False)
Tressp.shape


(8, 22)

BCI


In [60]:
Bci = reportes[
    (reportes["Campaña"] == "BCI")
]


Bci = Bci[(Bci["Empleador"] == Bci["Nombre Empresa"])]
Bci.to_excel("F:/Reportes SF/BCI/BCI_Base.xlsx", index=False)

Cenin = reportes[
    (reportes["Codigo EMP"] == "EMP6250")
]
Cenin = Cenin[(Cenin["Empleador"] == Cenin["Nombre Empresa"])]
Cenin.to_excel("F:/Reportes SF/BCI/Cenin_Base.xlsx", index=False)

Orbis = reportes[
    (reportes["Codigo EMP"] == "EMP6254")
    | (reportes["Codigo EMP"] == "EMP6255")

]
Orbis = Orbis[(Orbis["Empleador"] == Orbis["Nombre Empresa"])]
Orbis.to_excel("F:/Reportes SF/BCI/Orbis_Base.xlsx", index=False)

In [61]:
Colmena = reportes[
    (reportes["Campaña"] == "Colmena")
]


Colmena = Colmena[(Colmena["Empleador"] == Colmena["Nombre Empresa"])]
Colmena.to_excel("F:/Reportes SF/Colmena/Colmena_Base.xlsx", index=False)

In [62]:

Pedro = reportes[
    (reportes["Campaña"] == "Pedro De Valdivia")
]


Pedro = Pedro[(Pedro["Empleador"] == Pedro["Nombre Empresa"])]
Pedro.to_excel("F:/Reportes SF/Pedro de Valdivia/Pedro_Base.xlsx", index=False)

In [63]:

Vidacel = reportes[
    (reportes["Campaña"] == "Vidacel")
]


Vidacel = Vidacel[(Vidacel["Empleador"] == Vidacel["Nombre Empresa"])]
Vidacel.to_excel("F:/Reportes SF/Vidacel/Vidacel_Base.xlsx", index=False)

VidacelColab = reportes[
    (reportes["Campaña"] == "Vidacel Colaboradores")
]

VidacelColab = VidacelColab[(VidacelColab["Empleador"] == VidacelColab["Nombre Empresa"])]
VidacelColab.to_excel("F:/Reportes SF/Vidacel/VidacelColab_Base.xlsx", index=False)


In [64]:

Wfs = reportes[
    (reportes["Campaña"] == "Holding Wfs Food Services")
]

Wfs = Wfs[(Wfs["Empleador"] == Wfs["Nombre Empresa"])]
Wfs.to_excel("F:/Reportes SF/Wfs/Wfs_Base.xlsx", index=False)


In [65]:

Dermic = reportes[
    (reportes["Campaña"] == "Dermic")
]

Dermic = Dermic[(Dermic["Empleador"] == Dermic["Nombre Empresa"])]
Dermic.to_excel("F:/Reportes SF/Dermic/Dermic_Base.xlsx", index=False)
Dermic.shape

(92, 22)

In [66]:

Instruvalve = reportes[
    (reportes["Campaña"] == "Instruvalve")
]

Instruvalve = Instruvalve[(Instruvalve["Empleador"] == Instruvalve["Nombre Empresa"])]
Instruvalve.to_excel("F:/Reportes SF/Instruvalve/Instruvalve_Base.xlsx", index=False)

In [67]:

Thb = reportes[
    (reportes["Campaña"] == "THB Seguros")
]

Thb = Thb[(Thb["Empleador"] == Thb["Nombre Empresa"])]
Thb.to_excel("F:/Reportes SF/THB Seguros/Thb_Base.xlsx", index=False)

In [68]:
Metlife = reportes[
    (reportes["Campaña"] == "MetLife")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Metlife.to_excel("C:/Users/jmartinez/Documents/Metlife.xlsx", index=False)

Security = reportes[
    (reportes["Campaña"] == "Vida Security")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Security.to_excel("C:/Users/jmartinez/Documents/Vida Security.xlsx", index=False)

#Reportes_con_empresas.Empleador_x.fillna(Reportes_con_empresas['Nombre Empresa'], inplace=True)
reportes_francisca = Reportes_con_empresas
reportes_francisca.Empleador_x.fillna('Nones',inplace=True)
reportes_francisca = reportes_francisca[
    reportes_francisca['Empleador_x'] == 'Nones'
]
archivo = 'SinEmpresas.xlsx'
archivo1 = 'Empresas.xlsx'

fecha = str(datetime.date.today())
archivo_con_fecha = f"{fecha}_{archivo}"
archivo_con_fecha = archivo_con_fecha.replace(":", "_")
archivo_con_fecha = archivo_con_fecha.replace("-", "_")
archivo_con_fecha = archivo_con_fecha.replace(" ", "_")
#archivo_con_fecha = archivo_con_fecha.replace(".", "_")
hasta = Path("E:/Siempre/Usuarios/Francisca Rubio Yáñez/TI/Pacientes Sin Empresas/").joinpath(archivo_con_fecha)
print (hasta)
reportes_francisca.to_excel(
    hasta, index=False
)



E:\Siempre\Usuarios\Francisca Rubio Yáñez\TI\Pacientes Sin Empresas\2023_02_15_SinEmpresas.xlsx


In [69]:
fecha = str(datetime.date.today())
archivo_con_fecha = f"{fecha}_{archivo1}"
archivo_con_fecha = archivo_con_fecha.replace(":", "_")
archivo_con_fecha = archivo_con_fecha.replace("-", "_")
archivo_con_fecha = archivo_con_fecha.replace(" ", "_")
hasta = Path("E:/Siempre/Usuarios/Francisca Rubio Yáñez/TI/Empresas/").joinpath(archivo_con_fecha)
Programas_pacientes.to_excel(
    hasta, index=False
)

hasta = Path("F:/Empresas SF/empresas/").joinpath(archivo_con_fecha)
Programas_pacientes.to_excel(
    hasta, index=False
)